In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
df = pd.read_csv('steam_metadados.csv')
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [5]:
len(df.name.unique())

27033

In [10]:
def media_comprador(texto):
    txt_min, txt_max = texto.split('-')
    return (int(txt_max) + int(txt_min))/2

In [11]:
df['mean_owners'] = df['owners'].apply(media_comprador)

In [32]:
modelo = sm.OLS(df['price'], sm.add_constant(df[['required_age', 'average_playtime', 'positive_ratings']]))

In [30]:
sm.add_constant(df[['english', 'required_age', 'mean_owners', 'average_playtime', 'negative_ratings']]).corr()

,const,english,required_age,mean_owners,average_playtime,negative_ratings
const,NaN,NaN,NaN,NaN,NaN,NaN
english,NaN,1.000000,0.002860,0.010820,0.000338,0.005705
required_age,NaN,0.002860,1.000000,0.074438,0.034423,0.057599
mean_owners,NaN,0.010820,0.074438,1.000000,0.174157,0.690306
average_playtime,NaN,0.000338,0.034423,0.174157,1.000000,0.161140
negative_ratings,NaN,0.005705,0.057599,0.690306,0.161140,1.000000


In [33]:
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     156.2
Date:                Mon, 14 Nov 2022   Prob (F-statistic):          2.37e-100
Time:                        15:33:32   Log-Likelihood:                -94059.
No. Observations:               27075   AIC:                         1.881e+05
Df Residuals:                   27071   BIC:                         1.882e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                5.9054      0.048  

In [37]:
import plotly.express as px
df.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price',
       'mean_owners'],
      dtype='object')

In [71]:
def invroot(x):
    return 1/np.sqrt(x) if x > 0 else 0


In [72]:
df['achievements_inv'] = df['achievements'].apply(invroot)
df['average_playtime_inv'] = df['achievements'].apply(invroot)
df['positive_ratings_inv'] = df['positive_ratings'].apply(invroot)
df['negative_ratings_inv'] = df['negative_ratings'].apply(invroot)

In [73]:
modelo2 = sm.OLS(df['price'], sm.add_constant(df[['average_playtime_inv', 'positive_ratings_inv', 'achievements_inv', 'negative_ratings_inv']])).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     156.2
Date:                Mon, 14 Nov 2022   Prob (F-statistic):          2.37e-100
Time:                        15:51:41   Log-Likelihood:                -94059.
No. Observations:               27075   AIC:                         1.881e+05
Df Residuals:                   27071   BIC:                         1.882e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                5.9054      0.048  

In [74]:
px.scatter(df, y = 'price', x = 'average_playtime_inv')